In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import os
!pip install gitpython
import git
git.Repo.clone_from('https://github.com/gregorbachmann/scaling_mlps.git', '/kaggle/working/sample')
import sys

In [2]:
sys.path.append('/kaggle/working/sample')

In [3]:
torch.cuda.is_available()

True

In [4]:
import json
import torch

from sample.models import get_architecture
from sample.utils.parsers import get_finetune_parser
from sample.utils.config import config_to_name, model_from_config


# Path to pre-trained checkpoints and the corresponding config file (e.g. the checkpoints downloaded from given link)
checkpoint_path = '/kaggle/input/data-test-5/epoch_800'
config_path = '/kaggle/input/data-test-5/config.txt'

model, architecture, crop_resolution, norm = model_from_config(checkpoint_path)
with open(config_path, 'r') as f:
        config = json.load(f)

model = get_architecture(**config).cuda()
print('Loading checkpoint', checkpoint_path)

params = {
    k: v
    for k, v in torch.load(checkpoint_path).items()
}

# Load pre-trained parameters 
print('Load_state output', model.load_state_dict(params, strict=False))

Loading checkpoint /kaggle/input/data-test-5/epoch_800
Load_state output <All keys matched successfully>


In [5]:
model.eval()

BottleneckMLP(
  (act): GELU(approximate='none')
  (dropout): Identity()
  (linear_in): Linear(in_features=12288, out_features=512, bias=True)
  (linear_out): Linear(in_features=512, out_features=11230, bias=True)
  (blocks): ModuleList(
    (0-5): 6 x BottleneckBlock(
      (block): Sequential(
        (0): Linear(in_features=512, out_features=2048, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=2048, out_features=512, bias=True)
      )
    )
  )
  (layernorms): ModuleList(
    (0-5): 6 x LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
)